In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/default-of-credit-card-clients-dataset/UCI_Credit_Card.csv


In [21]:
import pandas as pd
import numpy as np

df = pd.read_csv('/kaggle/input/default-of-credit-card-clients-dataset/UCI_Credit_Card.csv')
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [22]:
df = df.drop(["ID"] , axis=1)

In [23]:
df.rename(columns={'PAY_0':'PAY_1'}, inplace=True)
df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,20000.0,2,2,1,24,2,2,-1,-1,-2,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,120000.0,2,2,2,26,-1,2,0,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,90000.0,2,2,2,34,0,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,50000.0,2,2,1,37,0,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,50000.0,1,2,1,57,-1,0,-1,0,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [24]:
#Checking the distribution in the Education category to infer some insights
df['EDUCATION'].value_counts()

EDUCATION
2    14030
1    10585
3     4917
5      280
4      123
6       51
0       14
Name: count, dtype: int64

In [25]:
df['MARRIAGE'].value_counts()

MARRIAGE
2    15964
1    13659
3      323
0       54
Name: count, dtype: int64

In [26]:
# Since in The Education field we have unknown as a category , we assume they are university graduates and map them to the same
df['EDUCATION'] = df['EDUCATION'].map({0:4,5:4,4:4,6:4,1:1,2:2,3:3})
df['MARRIAGE'] = df['MARRIAGE'].map({0:3,1:1,2:2,3:3})
#above mapping done due to discrepancy in data

In [27]:
#split the dataset into test & train sets
Y = df['default.payment.next.month']
X = df.drop(columns = ['default.payment.next.month'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42)

#scaling to bring down all the features to uniform range for better prediction
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

In [28]:
# we create a classifier model

from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
# Initialize the AdaBoost classifier
ada_boost = AdaBoostClassifier()

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],  # Number of estimators (default is 50)
    'learning_rate': [0.001, 0.01, 0.1, 1.0],  # Learning rate (default is 1.0)
    'algorithm': ['SAMME', 'SAMME.R']  # Algorithm to use (default is 'SAMME.R')
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=ada_boost, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')

# Fit GridSearchCV to the training data
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=AdaBoostClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['SAMME', 'SAMME.R'],
                         'learning_rate': [0.001, 0.01, 0.1, 1.0],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy')

In [29]:
# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best Parameters: {best_params}")
print(f"Best Score: {best_score}")

Best Parameters: {'algorithm': 'SAMME', 'learning_rate': 0.001, 'n_estimators': 50}
Best Score: 0.8196666666666668


In [31]:
adaClassifier = AdaBoostClassifier(n_estimators=50,algorithm='SAMME',learning_rate=0.001)
adaClassifier.fit(X_train_scaled,y_train)
from sklearn.model_selection import cross_val_score
scores = cross_val_score(adaClassifier, X_train_scaled, y_train, cv=5)
scores.mean()

0.8196666666666668

In [32]:
X_test_scaled = scaler.fit_transform(X_test)
y_predict = adaClassifier.predict(X_test_scaled)
y_predict

array([0, 0, 0, ..., 0, 0, 0])

In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_predict)

0.8193333333333334

In [34]:
data = {
    'Actual': y_test,
    'Predicted':y_predict
}

res = pd.DataFrame(data)
print(res['Predicted'].value_counts())
print(res['Actual'].value_counts())

Predicted
0    5393
1     607
Name: count, dtype: int64
Actual
0    4687
1    1313
Name: count, dtype: int64
